# Welcome to the Grid

In [1]:
type gridworld
  width::Int
  height::Int
  blockedStates::Array{Array{Int, 1}, 1}
  exitStates::Array{Array{Int, 1}, 1}
end

function states(g::gridworld)
  stateIndex = [[x, y] for x in 1:g.width for y in 1:g.height]
  stateIndex = filter(x->!(x in g.blockedStates), stateIndex)
  return stateIndex
end

function printGrid(g::gridworld)
  gridStates = states(g)
  mat = Array(Any, g.width, g.height)
  for (i, x) in enumerate(gridStates)
    mat[x[1], x[2]] = "$(x[1]),$(x[2])"
  end
  for (i, x) in enumerate(g.blockedStates)
    mat[x[1], x[2]] = "*"
  end
  for r in 1:size(mat, 1)
    for c in 1:size(mat, 2)
      @printf("%-4s   %s", mat[r,c], "")
    end
    print("\n")
  end
  println("Blocked: *")
  println("Exit states: $(join(["[$(x[1]),$(x[2])]" for x in g.exitStates], ", "))")
  return mat
end
;

In [5]:
g = gridworld(4,4, [[2,2], [3,3],[4,4]], [[3, 1]])
printGrid(g)
;

1,1    1,2    1,3    1,4    
2,1    *      2,3    2,4    
3,1    3,2    *      3,4    
4,1    4,2    4,3    *      
Blocked: *
Exit states: [3,1]


In [6]:
gridStates = states(g)

13-element Array{Array{Int64,1},1}:
 [1,1]
 [1,2]
 [1,3]
 [1,4]
 [2,1]
 [2,3]
 [2,4]
 [3,1]
 [3,2]
 [3,4]
 [4,1]
 [4,2]
 [4,3]

In [13]:
function randomPolicy(pos::Array{Int, 1}, g::gridworld)
  policy = zeros(size(states(g), 1))
  i,j = pos
  if (pos in g.exitStates) # absorbent case
    ind = find([x == pos for x in states(g)])
    policy[ind] = 1
  else # non absorbent
    nbs = [[i-1,j], [i+1,j], [i,j-1], [i,j+1]]
    nbs = filter(x -> x in states(g), nbs)
    n_nbs = size(nbs, 1)
    for x in nbs
      ind = find([state == x for state in states(g)])
      policy[ind] = 1/n_nbs
    end
  end
  return policy, states(g)
end
;

In [14]:
print(randomPolicy([1,1], g))

([0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],Array{Int64,1}[[1,1],[1,2],[1,3],[1,4],[2,1],[2,3],[2,4],[3,1],[3,2],[3,4],[4,1],[4,2],[4,3]])

In [15]:
print(randomPolicy([1,4], g))


([0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0],Array{Int64,1}[[1,1],[1,2],[1,3],[1,4],[2,1],[2,3],[2,4],[3,1],[3,2],[3,4],[4,1],[4,2],[4,3]])

In [16]:
print(randomPolicy([2,1], g))


([0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0],Array{Int64,1}[[1,1],[1,2],[1,3],[1,4],[2,1],[2,3],[2,4],[3,1],[3,2],[3,4],[4,1],[4,2],[4,3]])

In [17]:
print(randomPolicy([3,1], g))

([0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0],Array{Int64,1}[[1,1],[1,2],[1,3],[1,4],[2,1],[2,3],[2,4],[3,1],[3,2],[3,4],[4,1],[4,2],[4,3]])

In [18]:

function policyEval(policy::Function, g::gridworld; niter = 50)
  gridStates = states(g)
  transMat = reduce(vcat, [transpose(policy(x, g)[1]) for x in gridStates])
  rewardVect = map(x -> (x in g.exitStates) ? 0 : -1, gridStates)
  v = zeros(size(gridStates,1))
  for i in 1:niter
    v = rewardVect + transMat*v
  end
  return v
end


function valueGrid(val::Array{Float64, 1}, g::gridworld)
  gridStates = states(g)
  mat = Array(Any, g.width, g.height)
  for (i, x) in enumerate(gridStates)
    mat[x[1], x[2]] = val[i]
  end
  for (i, x) in enumerate(g.blockedStates)
    mat[x[1], x[2]] = "*"
  end
  println("Blocked: *")
  println("Exit states: $(join(["[$(x[1]),$(x[2])]" for x in g.exitStates], ", "))")
  return mat
end
;

In [32]:
g = gridworld(6,4, [[2,2], [3,3],[4,4]], [ [1,4]])
val = policyEval(randomPolicy, g, niter = 1000000)
valueGrid(val, g)


Blocked: *
Exit states: [1,4]


6×4 Array{Any,2}:
 -120.5     -79.5     -36.5       0.0  
 -159.5        "*"    -27.0     -15.5  
 -196.5    -210.597      "*"    -16.5  
 -216.403  -222.694  -229.421      "*" 
 -227.015  -230.355  -234.148  -236.779
 -231.288  -233.561  -236.039  -237.409

In [33]:

function greedyArrowGrid(val::Array{Float64, 1}, g::gridworld)
  valGrid = valueGrid(val, g)
  arrowGrid = Array(Any, g.width, g.height)
  gridStates = states(g)
  for s in gridStates
    i, j = s
    if s in g.exitStates
      arrowGrid[i,j] = ""
    else
      arrow = ""
      nbs = [[i-1,j], [i+1,j], [i,j-1], [i,j+1]]
      nbsVals = map(x -> (x in gridStates) ? round(valGrid[x...], 2) : -Inf, nbs)
      maxVal = maximum(nbsVals)
      maxInd = find([x == maxVal for x in nbsVals])
      for ind in maxInd
        if ind == 1
          arrow *= "↑"
        elseif ind == 2
          arrow *= "↓"
        elseif ind == 3
          arrow *= "←"
        elseif ind == 4
          arrow *= "→"
        end
      end
      arrowGrid[i,j] = arrow
    end
  end
  for x in g.blockedStates
    arrowGrid[x...] = "Blocked"
  end
  for x in g.exitStates
    arrowGrid[x...] = "Exit"
  end
  return arrowGrid
end
;

In [49]:

g = gridworld(5,5, [[2,1], [2,2], [3,3], [1,3]], [[3, 1], [5,5]])
val = policyEval(randomPolicy, g, niter = 10000)
greedyArrowGrid(val, g)


Blocked: *
Exit states: [3,1], [5,5]


5×5 Array{Any,2}:
 "→"        "←"        "Blocked"  "↓"  "↓"   
 "Blocked"  "Blocked"  "→"        "↓"  "↓"   
 "Exit"     "←"        "Blocked"  "↓"  "↓"   
 "↑"        "↑"        "←"        "↓"  "↓"   
 "↑"        "↑"        "→"        "→"  "Exit"

In [50]:
valueGrid(val, g)

Blocked: *
Exit states: [3,1], [5,5]


5×5 Array{Any,2}:
 -10000.0     -10000.0         "*"    -35.2312  -34.9554
       "*"          "*"     -34.5069  -33.5069  -32.6797
      0.0         -8.02437     "*"    -27.6098  -26.5768
    -10.4101     -14.0487   -17.8076  -19.7458  -16.4408
    -14.1815     -15.9529   -16.6284  -13.1247    0.0   

In [40]:
g = gridworld(6,6, [], [[6,6]])
val = policyEval(randomPolicy, g)
greedyArrowGrid(val, g)


Blocked: *
Exit states: [6,6]


6×6 Array{Any,2}:
 "↓→"  "→"   "→"   "→"   "↓"   "↓"   
 "↓"   "↓→"  "↓"   "↓"   "↓"   "↓"   
 "↓"   "→"   "↓→"  "↓"   "↓"   "↓"   
 "↓"   "→"   "→"   "↓→"  "↓"   "↓"   
 "→"   "→"   "→"   "→"   "↓→"  "↓"   
 "→"   "→"   "→"   "→"   "→"   "Exit"

In [41]:
g = gridworld(9,11, [[2,2], [3,3], [6,3]], [[3, 1], [6,2], [1,11]])
val = policyEval(randomPolicy, g)
greedyArrowGrid(val, g)

Blocked: *
Exit states: [3,1], [6,2], [1,11]


9×11 Array{Any,2}:
 "↓"     "←"        "←"        "←"  "←"  "→"  "→"  "→"  "→"  "→"  "Exit"
 "↓"     "Blocked"  "↑"        "←"  "←"  "→"  "→"  "→"  "→"  "↑"  "↑"   
 "Exit"  "←"        "Blocked"  "↓"  "←"  "←"  "→"  "→"  "↑"  "↑"  "↑"   
 "↑"     "↑"        "←"        "←"  "←"  "←"  "↑"  "↑"  "↑"  "↑"  "↑"   
 "↑"     "↓"        "←"        "←"  "←"  "←"  "←"  "↑"  "↑"  "↑"  "↑"   
 "→"     "Exit"     "Blocked"  "↑"  "←"  "←"  "←"  "↑"  "↑"  "↑"  "↑"   
 "↑"     "↑"        "←"        "←"  "←"  "←"  "←"  "↑"  "↑"  "↑"  "↑"   
 "↑"     "↑"        "←"        "←"  "←"  "←"  "←"  "←"  "↑"  "↑"  "↑"   
 "↑"     "↑"        "←"        "←"  "←"  "←"  "←"  "←"  "←"  "↑"  "↑"   

In [42]:
g = gridworld(4,4, [[2,2], [3,3]], [[4,4]])
val = policyEval(randomPolicy, g)
greedyArrowGrid(val, g)

Blocked: *
Exit states: [4,4]


4×4 Array{Any,2}:
 "↓→"  "→"        "↓→"       "↓"   
 "↓"   "Blocked"  "→"        "↓"   
 "↓→"  "↓"        "Blocked"  "↓"   
 "→"   "→"        "→"        "Exit"

In [53]:

function randomPolicy2(pos::Array{Int, 1}, g::gridworld)
  actions = states(g)
  policy = zeros(size(actions, 1))
  i,j = pos
  if (pos in g.exitStates) # absorbent case
    ind = find([x == pos for x in actions])
    policy[ind] = 1
  else # non absorvent
    nbs = [[i-1,j], [i+1,j], [i,j-1], [i,j+1],
            [i-1,j-1], [i+1,j-1], [i+1,j+1], [i-1,j+1]]
    nbs = filter(x -> x in actions, nbs)
    n_nbs = size(nbs, 1)
    for x in nbs
      ind = find([state == x for state in actions])
      policy[ind] = 1/n_nbs
    end
  end
  return policy, actions
end



function greedyArrowGrid2(val::Array{Float64, 1}, g::gridworld)
  valGrid = valueGrid(val, g)
  arrowGrid = Array(Any, g.width, g.height)
  gridStates = states(g)
  for s in gridStates
    i, j = s
    if s in g.exitStates
      arrowGrid[i,j] = ""
    else
      arrow = ""
      nbs = [[i-1,j], [i+1,j], [i,j-1], [i,j+1],
              [i-1,j-1], [i+1,j-1], [i+1,j+1], [i-1,j+1]]
      nbsVals = map(x -> (x in gridStates) ? round(valGrid[x...], 2) : -Inf, nbs)
      maxVal = maximum(nbsVals)
      maxInd = find([x == maxVal for x in nbsVals])
      for ind in maxInd
        if ind == 1
          arrow *= "↑"
        elseif ind == 2
          arrow *= "↓"
        elseif ind == 3
          arrow *= "←"
        elseif ind == 4
          arrow *= "→"
        elseif ind == 5
          arrow *= "↖"
        elseif ind == 6
          arrow *= "↙"
        elseif ind == 7
          arrow *= "↘"
        elseif ind == 8
          arrow *= "↗"
        end
      end
      arrowGrid[i,j] = arrow
    end
  end
  for x in g.blockedStates
    arrowGrid[x...] = "Blocked"
  end
  for x in g.exitStates
    arrowGrid[x...] = "Exit"
  end
  return arrowGrid
end
;

In [58]:
g = gridworld(5,5, [[2,1], [2,2], [3,3], [1,3]], [[3, 1], [5,5]])
val = policyEval(randomPolicy, g)
greedyArrowGrid2(val, g)

Blocked: *
Exit states: [3,1], [5,5]


5×5 Array{Any,2}:
 "→"        "↘"        "Blocked"  "↘"  "↓"   
 "Blocked"  "Blocked"  "↙"        "↘"  "↓"   
 "Exit"     "←"        "Blocked"  "↘"  "↓"   
 "↑"        "↖"        "↖"        "↘"  "↓"   
 "↑"        "↖"        "→"        "→"  "Exit"

In [59]:
valueGrid(val, g)

Blocked: *
Exit states: [3,1], [5,5]


5×5 Array{Any,2}:
 -50.0      -50.0        "*"   -29.5046  -29.282 
    "*"        "*"    -28.952  -28.1638  -27.476 
   0.0       -7.3975     "*"   -23.5074  -22.5775
  -9.60023  -12.8439  -15.97   -17.251   -14.1901
 -13.0517   -14.588   -15.028  -11.705     0.0   

In [46]:
g = gridworld(6,6, [], [[6, 6]])
val = policyEval(randomPolicy2, g, niter = 1000)
greedyArrowGrid(val, g)

Blocked: *
Exit states: [6,6]


6×6 Array{Any,2}:
 "↘"  "↘"  "↘"  "↘"  "↘"  "↓"   
 "↘"  "↘"  "↘"  "↘"  "↘"  "↓"   
 "↘"  "↘"  "↘"  "↘"  "↘"  "↓"   
 "↘"  "↘"  "↘"  "↘"  "↘"  "↓"   
 "↘"  "↘"  "↘"  "↘"  "↘"  "↓"   
 "→"  "→"  "→"  "→"  "→"  "Exit"

In [48]:
g = gridworld(9,11, [[2,2], [3,3], [6,3]], [[3, 1], [6,2], [1,11]])
val = policyEval(randomPolicy, g)
greedyArrowGrid(val, g)

Blocked: *
Exit states: [3,1], [6,2], [1,11]


9×11 Array{Any,2}:
 "↓"     "↙"        "←"        "←"  "←"  "→"  "→"  "→"  "→"  "→"  "Exit"
 "↓"     "Blocked"  "↙"        "↖"  "↖"  "↗"  "↗"  "↗"  "↗"  "↗"  "↑"   
 "Exit"  "←"        "Blocked"  "↙"  "↙"  "↙"  "↗"  "↗"  "↗"  "↗"  "↑"   
 "↑"     "↖"        "↖"        "↙"  "←"  "←"  "↗"  "↗"  "↗"  "↗"  "↑"   
 "↘"     "↓"        "↙"        "←"  "↖"  "↖"  "↖"  "↗"  "↗"  "↗"  "↑"   
 "→"     "Exit"     "Blocked"  "↖"  "↖"  "↖"  "↖"  "↖"  "↗"  "↗"  "↑"   
 "↗"     "↑"        "↖"        "←"  "←"  "↖"  "↖"  "↖"  "↖"  "↗"  "↑"   
 "↑"     "↖"        "↖"        "↖"  "↖"  "↖"  "↖"  "↖"  "↖"  "↖"  "↑"   
 "↑"     "↖"        "↖"        "↖"  "↖"  "↖"  "↖"  "↖"  "↖"  "↖"  "↖"   